<a href="https://colab.research.google.com/github/iraidaantropova/ADD/blob/main/ADD_HW8_SinitsaI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Урок 8. Снижение размерности данных

Домашнее задание:

Используя файл Lesson_8_extended.ipynb (он в web8.zip в материалах):

Обучить любую модель классификации на датасете IRIS до применения PCA (2 компоненты) и после него. Сравнить качество классификации по отложенной выборке.


*Написать свою реализацию метода главных компонент с помощью сингулярного разложения с использованием функции numpy.linalg.svd()


In [19]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt
import random
from sklearn import model_selection

In [48]:
def pca(X, n_components):
    # Центрирование данных
    X = X - np.mean(X, axis=0)
    
    # Вычисление матрицы ковариации данных
    cov_matrix = np.cov(X.T)
    
    # Вычисление сингулярного разложения матрицы ковариации данных
    U, S, V = np.linalg.svd(cov_matrix)
    
    # Сортировка сингулярных значений в порядке убывания и соответствующих им сингулярных векторов
    idx = np.argsort(-S)
    U = U[:, idx]
    S = S[idx]
    
    # Выбор наиболее значимых сингулярных векторов
    U_reduced = U[:, :n_components]
    
    # Умножение матрицы данных на выбранные сингулярные векторы
    X_reduced = np.dot(X, U_reduced)
    
    return X_reduced

In [20]:
# Генерую N-бутстрап выборки

def get_bootstrap(data, labels, N):
    random.seed(42)
    n_samples = data.shape[0]
    bootstrap = []
    
    for i in range(N):
        b_data = np.zeros(data.shape)
        b_labels = np.zeros(labels.shape)
        for j in range(n_samples):
            sample_index = random.randint(0, n_samples-1)
            b_data[j] = data[sample_index]
            b_labels[j] = labels[sample_index]            
        bootstrap.append((b_data, b_labels))
        
    return bootstrap

In [21]:
def get_subsample(len_sample):
    # оставляю не признаки, а их индексы
    sample_indexes = [i for i in range(len_sample)]
    
    len_subsample = int(np.sqrt(len_sample))
    subsample = []
    
    random.shuffle(sample_indexes)
    for _ in range(len_subsample):
        subsample.append(sample_indexes.pop())
        
    return subsample

In [22]:
# Реализация класса узла

class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

In [23]:
# И класс терминального листа

class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
        
    def predict(self):
        # подсчет количества объектов разных классов
        classes = {}  # формирование словаря "класс: количество объектов"
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
        #  поиск класса, количество объектов которого будет максимальным в этом листе и возвращение его    
        prediction = max(classes, key=classes.get)
        return prediction       

In [24]:
# Критерий Джини

def gini(labels):
    #  подсчет количества объектов разных классов
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1
    
    #  расчет критерия
    impurity = 1
    for label in classes:
        p = classes[label] / len(labels)
        impurity -= p ** 2
        
    return impurity

In [25]:
# Расчет качества

def quality(left_labels, right_labels, current_gini):

    # доля выбоки, ушедшая в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    
    return current_gini - p * gini(left_labels) - (1 - p) * gini(right_labels)

In [26]:
# Разбиение датасета в узле

def split(data, labels, index, t):
    
    left = np.where(data[:, index] <= t)
    right = np.where(data[:, index] > t)
        
    true_data = data[left]
    false_data = data[right]
    true_labels = labels[left]
    false_labels = labels[right]
        
    return true_data, false_data, true_labels, false_labels

In [27]:
# Поиск наилучшего разбиения

def find_best_split(data, labels):
    
    # фиксируем минимальное количество объектов в узле
    min_leaf = 5

    current_gini = gini(labels)

    best_quality = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    # выбор индекса из подвыборки длиной sqrt(n_features)
    subsample = get_subsample(n_features)
    
    for index in subsample:
        # проверка только уникальных значений признака, исключая повторения
        t_values = np.unique([row[index] for row in data])
        
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            #  пропуск разбиения, в которых в узле остается менее 5 объектов
            if len(true_data) < min_leaf or len(false_data) < min_leaf:
                continue
            
            current_quality = quality(true_labels, false_labels, current_gini)
            
            #  выбор порога, на котором получается максимальный прирост качества
            if current_quality > best_quality:
                best_quality, best_t, best_index = current_quality, t, index

    return best_quality, best_t, best_index

In [28]:
# Построение дерева с помощью рекурсивной функции

def build_tree(data, labels):

    quality, t, index = find_best_split(data, labels)

    #  Базовый случай - прекращение рекурсии, когда нет прироста в качества
    if quality == 0:
        return Leaf(data, labels)

    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)

    # Рекурсивно строю два поддерева
    true_branch = build_tree(true_data, true_labels)
    false_branch = build_tree(false_data, false_labels)

    # Возвращаю класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

In [29]:
def random_forest(data, labels, n_trees):
    forest = [] # список деревьев
    bootstrap = get_bootstrap(data, labels, n_trees) # создание n_trees бутстреп выборок
    
    for b_data, b_labels in bootstrap:
        forest.append(build_tree(b_data, b_labels)) # добавление по дереву в ансамбль
    return forest

In [30]:
# Функция классификации отдельного объекта

def classify_object(obj, node):

    #  Остановка рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)

In [31]:
# функция формирования предсказания по выборке на одном дереве

def predict(data, tree):
    
    classes = []
    for obj in data:
        prediction = classify_object(obj, tree)
        classes.append(prediction)
    return classes

In [32]:
# предсказание голосованием деревьев

def tree_vote(forest, data):

    # добавление предсказания деревьев в список
    predictions = []
    for tree in forest:
        predictions.append(predict(data, tree))
    
    # формирование списка с предсказаниями для каждого объекта
    predictions_per_object = list(zip(*predictions))
    
    # выбор в качестве итогового предсказания для каждого объекта то,
    # за которое проголосовало большинство деревьев
    voted_predictions = []
    for obj in predictions_per_object:
        voted_predictions.append(max(set(obj), key=obj.count))
        
    return voted_predictions

In [35]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
# Загружаем датасет IRIS
iris = load_iris()
X = iris.data
y = iris.target

In [36]:
# Ввод функции подсчета точности как доли правильных ответов

def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

In [44]:
# Разделение выборки на обучающую и тестовую, и в итоге получается точность модели на исходных данных:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [45]:
def metrics_random_forest(X_train, y_train, n_trees):
    my_forest=random_forest(X_train, y_train, n_trees)
    # ответы для обучающей выборки
    y_pred = tree_vote(my_forest, X_test)
    # точность на обучающей выборке
    train_accuracy=accuracy_metric(y_test, y_pred)
    #точность на тестовой выборке
    test_accuracy = accuracy_metric(X_test,y_test)
    return test_accuracy

In [47]:
my_forest=random_forest(X_train, y_train, 15)
y_pred = tree_vote(my_forest, X_test)
accuracy_metric(y_test, y_pred)

97.77777777777777

In [49]:
# Применим PCA для уменьшения размерности данных
X_reduced = pca(X, 2)

In [53]:
# Разделение выборки на обучающую и тестовую, и в итоге получается точность модели на исходных данных:

X_train, X_test, y_train, y_test = train_test_split(X_reduced, y, test_size=0.3, random_state=42)

In [54]:
my_forest=random_forest(X_train, y_train, 15)
y_pred = tree_vote(my_forest, X_test)
accuracy_metric(y_test, y_pred)

95.55555555555556

после применения PCA качество модели ухудшилось